In [ ]:
#default_exp text_norm

# Text Normalization
> Functions used for TTS Dataset Preparation

In [ ]:
#export
import re
from fastcore.test import *
from nbdev.showdoc import *

In [ ]:
#export
_whitespace_re = re.compile(r'\s+')

def collapse_whitespace(text):
    "Replace multiple whitespaces with single space."
    return re.sub(_whitespace_re, ' ', text)

In [ ]:
test_eq(collapse_whitespace(
    " 1  2   3     4     5       "),
    " 1 2 3 4 5 ")

In [ ]:
#export
def lowercase(text):
    return text.lower()

In [ ]:
test_eq(lowercase(
        'ПрИвеТ ЧуВАК!'), 
        'привет чувак!')

In [ ]:
#export
def check_no_numbers(text):
    "Return a list of digits, or empty, if not found."
    return re.findall(r"(\d+)", text)

In [ ]:
#export
_specials = [(re.compile(f'{x[0]}'), x[1]) for x in [
  ('!\.', '!'),
  ('\?\.', '?'),
  ('\/', ''),
  ('…', '.'),
  ('\.\.', '.')
]]

In [ ]:
#export
def remove_specials(text):
    for regex, replacement in _specials:
        text = re.sub(regex, replacement, text)
    return text

In [ ]:
test_eq(remove_specials('Ой!. Ага?. Пауза./ Стоп.. Многоточие…'),'Ой! Ага? Пауза. Стоп. Многоточие.')

In [ ]:
#export
_abbreviations = [(re.compile(f'\\b{x[0]}', re.IGNORECASE), x[1]) for x in [
  ('т.е.', 'то есть'),
  ('т.к.', 'так как'),
  ('и т.д.', 'и так далее'),
  ('и т.п.', 'и тому подобное')
]]

In [ ]:
#export
def expand_abbreviations(text):
    for regex, replacement in _abbreviations:
        text = re.sub(regex, replacement, text)
    return text

In [ ]:
test_eq(expand_abbreviations(\
            'Привет Джон, т.е. Иван. Т.к. русский. И т.д. И т.п.'),\
            'Привет Джон, то есть Иван. так как русский. и так далее и тому подобное') 

In [ ]:
#export
def basic_cleaner(text):
    "Basic pipeline that lowercases and collapses whitespaces."
    text = lowercase(text)
    text = collapse_whitespace(text)
    return text

In [ ]:
test_eq(basic_cleaner(
    'Привет   Джон, т.е. Иван, т.к. русский. И т.д. и т.п.'),
    'привет джон, т.е. иван, т.к. русский. и т.д. и т.п.')

In [ ]:
#export
def russian_cleaner(text):
    """Pipeline for Russian text cleaning: 
       lowercase, remove_specials, expand_abbreviations, collapse_whitespace, check_no_numbers."""
    text = lowercase(text)
    text = remove_specials(text)
    text = expand_abbreviations(text)
    text = collapse_whitespace(text)
    numbers = check_no_numbers(text)
    if (numbers != []): 
        print("Запишите числительные не цифрами, а словами:")
        print(numbers)
    return text

In [ ]:
test_eq(russian_cleaner(
        'Привет Джон, т.е.     Иван, т.к. русский. И т.д. и т.п. Ой!. Ага?. / Стоп..'),
        'привет джон, то есть иван, так как русский. и так далее и тому подобное ой! ага? стоп.')

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 01_text_norm.ipynb.
Converted index.ipynb.
